In [ ]:
import pickle
import numpy as np
from tensorflow.keras.models import load_model
import pretty_midi
import random

In [ ]:
# ===== Load the model and data =====
with open("D:\Study\Ai_Music_Composer\project\data\generator_training_data.pkl", "rb") as f:
    data = pickle.load(f)

X_notes = data["X_notes"]
X_moods = data["X_moods"]
note_to_idx = data["note_to_idx"]
idx_to_note = data["idx_to_note"]
mood_encoder = data["mood_encoder"]

model = load_model("D:\Study\Ai_Music_Composer\project\models\music_generator.h5")

num_moods = len(mood_encoder)

In [ ]:
# ===== Temperature sampling =====
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-9) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

In [ ]:
# ===== Music generation =====
def generate_notes(model, start_notes, mood_label, num_notes=50, temperature=0.8):
    generated = list(start_notes)

    # One-hot encode the mood
    mood_input = np.zeros((1, len(mood_encoder)))
    mood_input[0, mood_label] = 1

    for _ in range(num_notes):
        input_seq = np.array(generated[-50:])  # last 50 notes
        if len(input_seq) < 50:
            input_seq = np.pad(input_seq, (50 - len(input_seq), 0), mode='constant')

        input_seq = np.expand_dims(input_seq, axis=0)
        
        preds = model.predict([input_seq, mood_input], verbose=0)
        next_idx = sample_with_temperature(preds[0], temperature)
        generated.append(next_idx)

    return generated

In [ ]:
# ===== Convert notes to MIDI =====
def notes_to_midi(note_indices, output_file):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    start = 0
    for idx in note_indices:
        note_number = idx_to_note[idx]
        note = pretty_midi.Note(
            velocity=100, pitch=note_number, start=start, end=start+0.5
        )
        instrument.notes.append(note)
        start += 0.5
    midi.instruments.append(instrument)
    midi.write(output_file)

In [ ]:
# ===== Main generation for each mood =====
moods = mood_encoder
for mood_idx, mood_name in enumerate(moods):
    print(f"Generating music for mood: {mood_name}")
    start_notes = [60, 64, 67]  # C major chord as seed
    generated_notes = generate_notes(model, start_notes, mood_idx, num_notes=100, temperature=0.9)
    output_file = f"generated_{mood_name}.mid"
    notes_to_midi(generated_notes, output_file)
    print(f"Saved: {output_file}")